In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import tifffile
import rasterio

In [6]:
data = np.loadtxt('/home/hzin/earthAI/logs/3d_140102.txt')
print(data.shape)

(89521, 9)


In [7]:
lat, lon = data[:, 0], data[:, 1]
min_lat = min(lat)
max_lat = max(lat)
min_lon = min(lon)
max_lon = max(lon)
print(min_lat, max_lat, min_lon, max_lon)

20.000358595371228 41.99959794505977 -104.99896537745376 -85.00068646989837


In [8]:
resolution = 0.05
grid_lat, grid_lon = np.meshgrid(
    np.arange(min_lat, max_lat, resolution),
    np.arange(min_lon, max_lon, resolution)
)

In [22]:
print(grid_lat.shape, grid_lon.shape)

(400, 440) (400, 440)


In [17]:
d0 = griddata(
        data[data[:, 8] == 0, :2],
        data[data[:, 8] == 0, 2:8],
        (grid_lat, grid_lon),
        method='nearest'
    )
d1 = griddata(
        data[data[:, 8] == 1, :2],
        data[data[:, 8] == 1, 2:8],
        (grid_lat, grid_lon),
        method='nearest'
    )
d2 = griddata(
        data[data[:, 8] == 2, :2],
        data[data[:, 8] == 2, 2:8],
        (grid_lat, grid_lon),
        method='nearest'
    )
print(d0.shape, d1.shape, d2.shape)


(400, 440, 6) (400, 440, 6) (400, 440, 6)


In [23]:
d = np.concatenate((d0, d1, d2), axis=2).transpose(2, 0, 1)
print(d.shape)

(18, 400, 440)


In [27]:
# Find the minimum and maximum values along the first three dimensions
min_vals = np.min(d, axis=(1, 2), keepdims=True)
max_vals = np.max(d, axis=(1, 2), keepdims=True)

# Rescale the values
rad_scaled = ((d - min_vals) / (max_vals - min_vals)) * 255

# Convert rescaled values to integers
rad_scaled = rad_scaled.astype(np.uint8)
print(rad_scaled.shape)

(18, 400, 440)


In [35]:
# Save the combined grid to a TIFF file
output_tiff_file = "140101_3d.tiff"
tifffile.imwrite(output_tiff_file, rad_scaled, photometric='rgb')